In [67]:
import highspy

In [68]:
class Cut():
    def __init__(self, var_id, lower, upper):
        self.var_id = var_id
        self.lower = lower
        self.upper = upper

    def __str__(self):
        return f"Cut[id: {self.var_id}, lower: {self.lower}, upper: {self.upper}]"

In [69]:
class StackItem():
    def __init__(self, cuts, solution, dual_value):
        self.cuts = cuts
        self.solution = solution.col_value
        self.dual_value = dual_value

    def __str__(self):
        text = "StackItem{\n\tCuts:\n"
        for cut in self.cuts:
            text += "\t\t" + str(cut) + "\n"
        text += "\tSolution:\n\t\t" + str(self.solution) + "\n"
        text += "\tDual value:\n\t\t" + str(self.dual_value) + "\n}"
        return text

In [71]:
class BnB:
    def __init__(self, solver, generalize, eps):
        self.__solver = solver
        self.__generalize = generalize
        self.__eps = eps

        self.__solver.run()
        self.__dual_value = self.__solver.getInfo().objective_function_value
        self.__stack = [StackItem([], self.__solver.getSolution(), self.__dual_value)]

    def start(self):
        stack_item = self.__stack.pop()
        
        while stack_item is not None:

            print(stack_item)
            
            if self.check_on_primal(stack_item.solution):
                print(stack_item.solution)
                
            left_cut, right_cut = self.find_cut(stack_item)
            print(left_cut, "|", right_cut)
            
            while left_cut is None and right_cut is None:
                if not self.__stack:
                    return
                stack_item = self.__stack.pop()
                left_cut, right_cut = self.find_cut(stack_item)
                self.__dual_value = stack_item.dual_value

            left_solver = highspy.Highs()
            left_solver.passModel(self.__solver.getModel())
            left_solver.silent()
            left_solver.changeColBounds(left_cut.var_id, left_cut.lower, left_cut.upper)
            
            right_solver = highspy.Highs()
            right_solver.passModel(self.__solver.getModel())
            right_solver.silent()
            right_solver.changeColBounds(right_cut.var_id, right_cut.lower, right_cut.upper)
            for cut in stack_item.cuts:
                left_solver.changeColBounds(cut.var_id, cut.lower, cut.upper)
                right_solver.changeColBounds(cut.var_id, cut.lower, cut.upper)

            left_solver.run()
            right_solver.run()

            left_objective_function_value = left_solver.getInfo().objective_function_value
            right_objective_function_value = right_solver.getInfo().objective_function_value

            if left_objective_function_value < right_objective_function_value:
                self.__dual_value = left_objective_function_value
                self.__stack += [StackItem(stack_item.cuts + [right_cut], right_solver.getSolution(), right_objective_function_value)]
                stack_item = StackItem(stack_item.cuts + [left_cut], left_solver.getSolution(), left_objective_function_value)
            else:
                self.__dual_value = right_objective_function_value
                self.__stack += [StackItem(stack_item.cuts + [left_cut], left_solver.getSolution(), left_objective_function_value)]
                stack_item = StackItem(stack_item.cuts + [right_cut], right_solver.getSolution(), right_objective_function_value)

    def check_on_primal(self, dual_value):
        for icol in self.__generalize:
            if abs(dual_value[icol] - (dual_value[icol] // 1)) > self.__eps:
                return False
        return True

    def find_cut(self, stack_item):
        heuristics = lambda value: abs(value - 0.5)
        res_id = 0
        res_func_value = self.__solver.inf
        lower = 0
        lower_res = 0
        find_one = False
        
        for icol in self.__generalize:
            value = stack_item.solution[icol]
            lower = value // 1
            
            if abs(lower - value) > self.__eps:
                val = heuristics(value - lower)
                if val < res_func_value:
                    res_id = icol
                    res_func_value = val
                    find_one = True
                    lower_res = int(lower)
        if find_one:
            return (Cut(res_id, 0, lower_res), Cut(res_id, lower_res + 1, self.__solver.inf))
        return (None, None)

In [77]:
help(h.getModelStatus())

Help on HighsModelStatus in module highspy._core object:

class HighsModelStatus(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      HighsModelStatus
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __eq__(...)
 |      __eq__(self: object, other: object) -> bool
 |
 |  __getstate__(...)
 |      __getstate__(self: object) -> int
 |
 |  __hash__(...)
 |      __hash__(self: object) -> int
 |
 |  __index__(...)
 |      __index__(self: highspy._core.HighsModelStatus) -> int
 |
 |  __init__(...)
 |      __init__(self: highspy._core.HighsModelStatus, value: int) -> None
 |
 |  __int__(...)
 |      __int__(self: highspy._core.HighsModelStatus) -> int
 |
 |  __ne__(...)
 |      __ne__(self: object, other: object) -> bool
 |
 |  __repr__(...)
 |      __repr__(self: object) -> str
 |
 |  __setstate__(...)
 |      __setstate__(self: highspy._core.HighsModelStatus, state: int) -> None
 |
 |  __str__(...)
 |      __str__(self: 

In [76]:
help(h)

Help on Highs in module highspy.highs object:

class Highs(highspy._core._Highs)
 |  HiGHS solver interface
 |
 |  Method resolution order:
 |      Highs
 |      highspy._core._Highs
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __init__(self)
 |      __init__(self: highspy._core._Highs) -> None
 |
 |  addBinaries(
 |      self,
 |      *nvars: 'Union[int, Mapping[Any, Any], Sequence[Any]]',
 |      **kwargs: 'Union[Any, HighsVarType, Mapping[Any, Any], Sequence[Any]]'
 |  )
 |      Alias for the addVariables method, for binary variables.
 |
 |  addBinary(self, obj: 'float' = 0.0, name: 'Optional[str]' = None)
 |      Alias for the addVariable method, for binary variables.
 |
 |  addConstr(self, expr: 'highs_linear_expression', name: 'Optional[str]' = None)
 |      Adds a constraint to the model.
 |
 |      Args:
 |          expr: A highs_linear_expression to be added.
 |          name: Optional name of the constraint.
 |
 |      

In [72]:
h = highspy.Highs()
h.readModel("test.lp");
h.silent()
B = BnB(h, [0], 1e-3)

Running HiGHS 1.9.0 (git hash: fa40bdf): Copyright (c) 2024 HiGHS under MIT licence terms


In [66]:
B.start()

StackItem{
	Cuts:
	Solution:
		[8.5, 0.0]
	Dual value:
		-8.5
}
Cut[id: 0, lower: 0, upper: 8] | Cut[id: 0, lower: 9, upper: inf]
Running HiGHS 1.9.0 (git hash: fa40bdf): Copyright (c) 2024 HiGHS under MIT licence terms
Running HiGHS 1.9.0 (git hash: fa40bdf): Copyright (c) 2024 HiGHS under MIT licence terms
StackItem{
	Cuts:
		Cut[id: 0, lower: 0, upper: 8]
	Solution:
		[8.0, 0.3333333333333333]
	Dual value:
		-8.333333333333334
}
Cut[id: 1, lower: 0, upper: 0] | Cut[id: 1, lower: 1, upper: inf]
Running HiGHS 1.9.0 (git hash: fa40bdf): Copyright (c) 2024 HiGHS under MIT licence terms
Running HiGHS 1.9.0 (git hash: fa40bdf): Copyright (c) 2024 HiGHS under MIT licence terms
StackItem{
	Cuts:
		Cut[id: 0, lower: 0, upper: 8]
		Cut[id: 1, lower: 1, upper: inf]
	Solution:
		[7.0, 1.0]
	Dual value:
		-8.0
}
[7.0, 1.0]
None | None
